# Clase Práctica #6 (Compilación)

In [9]:
import pydot

## Autómatas finitos

Un autómata finito (determinista o no determinista) es un quíntuplo $A = <Q,q_0,V,F,f>$ con las siguientes características:

* $Q$ es un conjunto finito de estados ($Q = \{ q_0, \ldots, q_n \}$), de ahí el adjetivo de **finito**.
* $q_0 \in Q$ es el estado inicial.
* $V$ es un conjunto finito de símbolos que pueden aparecer en la cinta.
* $F \subseteq Q$ es un subconjunto de estados que denominaremos *estados finales*.
* $f$ es una *función de transición*, que determina, para cada par posible de estados y símbolos, cuál es el estado de destino. En la forma de esta función radica justamente la diferencia entre AF determinista y no determinista:
    - $f: Q \times V \to Q$ denota un autómata **determinista** justamente porque en un estado particular, para un símbolo particular, existe solamente un estado posible de destino (o ninguno), por lo tanto, siempre existe una única decisión que tomar.
    - $f: Q \times (V \cup \{ \epsilon \}) \to 2^Q$ denota un autómata **no determinista** porque en un estado particular, para un símbolo particular, existen potencialmente múltiples estados de destino (o ninguno). Incluso permite realizar $\epsilon$-transiciones (transiciones que no consumen símbolos de la cinta) lo cual resalta aún más el carácter no determinista de estos autómatas.

### Pasando a Python

Modelemos los autómatas finitos no deterministas con la clase `NFA`. Dado que nunca los usaremos directamente para reconocer cadenas, estos funcionarán simplemente como contenedores de los estados y transiciones.

In [10]:
class NFA:
    def __init__(self, states, finals, transitions, start=0):
        self.states = states
        self.start = start
        self.finals = set(finals)
        self.map = transitions
        self.vocabulary = set()
        self.transitions = { state: {} for state in range(states) }
        
        for (origin, symbol), destinations in transitions.items():
            assert hasattr(destinations, '__iter__'), 'Invalid collection of states'
            self.transitions[origin][symbol] = destinations
            self.vocabulary.add(symbol)
            
        self.vocabulary.discard('')
        
    def epsilon_transitions(self, state):
        assert state in self.transitions, 'Invalid state'
        try:
            return self.transitions[state]['']
        except KeyError:
            return ()
            
    def graph(self):
        G = pydot.Dot(rankdir='LR', margin=0.1)
        G.add_node(pydot.Node('start', shape='plaintext', label='', width=0, height=0))

        for (start, tran), destinations in self.map.items():
            tran = 'ε' if tran == '' else tran
            G.add_node(pydot.Node(start, shape='circle', style='bold' if start in self.finals else ''))
            for end in destinations:
                G.add_node(pydot.Node(end, shape='circle', style='bold' if end in self.finals else ''))
                G.add_edge(pydot.Edge(start, end, label=tran, labeldistance=2))

        G.add_edge(pydot.Edge('start', self.start, label='', style='dashed'))
        return G

    def _repr_svg_(self):
        try:
            return self.graph().create_svg().decode('utf8')
        except:
            pass

Los parámetros del constructor tienen la siguente semántica:
- `states`: representa en número de estados del autómata. Los estados se modelarán como números, comenzando en `0` y hasta `states-1`.
- `finals`: representa la colección de estados finales del autómata. Dado que los estados se representan con números, este debe ser una colección de números.
- `transitions`: representa la función de transición. Se espera un diccionario que, dados como llaves un estado origen (un número) y un símbolo (un string), devuelve como valor una colección de estados destino (números). Para renotar una $\epsilon$-transición usaremos el string vacío.
- `start`: representa el estado inicial del autómata.

Por ejemplo, construyamos un autómata que reconozca el lenguaje de las cadenas sobre `{a,b}*` que terminan en `ba`.

In [11]:
nfa = NFA(states=3, finals=[2], transitions={
    (0, 'a'): [0],
    (0, 'b'): [0, 1],
    (1, 'a'): [2]
})
nfa

![](cp6-files/0.svg)

Una vez construida la instancia las transiciones pueden accederse de forma más cómoda a través del campo `transitions`. Las transiciones están agrupadas por estado origen, y luego por símbolo: $(Q \to (V \to Q))$.

In [12]:
nfa.transitions

{0: {'a': [0], 'b': [0, 1]}, 1: {'a': [2]}, 2: {}}

### Autómatas finitos deterministas

Modelemos los autómatas finitos deterministas con la clase `DFA`. Esta clase extiende la clase `NFA` para:
- Usar la función de transición propia de los autómatas finitos deterministas.
- Implementar un algoritmo de reconocimiento de cadenas.

In [13]:
class DFA(NFA):
    
    def __init__(self, states, finals, transitions, start=0):
        assert all(isinstance(value, int) for value in transitions.values())
        assert all(len(symbol) > 0 for origin, symbol in transitions)
        
        transitions = { key: [value] for key, value in transitions.items() }
        NFA.__init__(self, states, finals, transitions, start)
        self.current = start
        
    def _move(self, symbol):
        try:
            self.current = self.transitions[self.current][symbol][0]
        except KeyError:
            self.current = -1
    
    def _reset(self):
        self.current = self.start
        
    def recognize(self, string):
        self._reset()
        for symbol in string:
            self._move(symbol)
            if self.current == -1:
                return False
        return self.current in self.finals


La instancia se construye de forma similar a los NFA, salvo por la forma de la función de transición que, en lugar de una colección de estados destino, contiene un único estado destino.

In [14]:
automaton = DFA(states=3, finals=[2], transitions={
    (0, 'a'): 0,
    (0, 'b'): 1,
    (1, 'a'): 2,
    (1, 'b'): 1,
    (2, 'a'): 0,
    (2, 'b'): 1,
})
automaton

![](cp6-files/1.svg)

El autómata anterior reconoce el lenguaje de las cadenas sobre `{a,b}*` que terminan en `ba`.

In [15]:
assert automaton.recognize('ba')
assert automaton.recognize('aababbaba')

assert not automaton.recognize('')
assert not automaton.recognize('aabaa')
assert not automaton.recognize('aababb')

## Equivalencia entre DFA y NFA

Como estudiamos en conferencia, los autómatas finitos deterministas y no deterministas tienen el mismo poder de reconocimiento: ambos reconocen la familia de lenguajes regulares. Dicha característica nos indica que deberían ser transformables los unos a los otros.

La transformación de DFA a NFA es bastante trivial, ya que un autómata finito determinista puede verse como uno no deterministas sin $\epsilon$-transiciones y en el que los conjuntos de estados destinos en las respectivas transiciones siempre tiene tamaño 1 (o 0). De hecho, eso es lo que hacemos en el constructor de la clase `DFA` que hereda de `NFA`.

La transformación de NFA a DFA se basa en simular las transiciones entre los posibles estados en los que puede estar a la vez el NFA durante el reconocimiento de cualquier cadena. Los estados de la versión transformada del NFA serán cada uno un subconjunto del conjunto de estados original. Sabemos que el número de posibles configuraciones de estados en los que puede estar a la vez es finito, ya que el número de estados del autómata es finito. Un autómata con $N$ estados puede tener a lo sumo $2^N$ posibles configuraciones de estados distintas.

El algoritmo clásico para convertir de NFA a DFA se apoya en 2 definiciones auxiliares:
- `move(T,a)`: conjunto de estados del NFA hacia los que hay una transición con el símbolo de entrada `a` desde algún estado de `T`.
- $\epsilon$-closure`(T)`: conjunto de estados del NFA alcanzables desde algún estado de `T` realizando únicamente $\epsilon$-transiciones.

In [16]:
from cmp.utils import ContainerSet

Trabajaremos con el siguiente autómata de ejemplo, que reconoce el lenguaje `c*|(a|b)d*`.

In [17]:
automaton = NFA(states=6, finals=[3, 5], transitions={
    (0, ''): [ 1, 2 ],
    (1, ''): [ 3 ],
    (1,'b'): [ 4 ],
    (2,'a'): [ 4 ],
    (3,'c'): [ 3 ],
    (4, ''): [ 5 ],
    (5,'d'): [ 5 ]
})
automaton

![](cp6-files/2.svg)

Pasemos a computar `move(T,a)`. El método `move(automaton, states, symbol)` computa el conjunto `move(states, symbol)` con las transiciones registradas en `automaton`.

In [19]:
def move(automaton, states, symbol):
    moves = set()
    for state in states:
        try:
            moves.update(automaton.transitions[state][symbol])
        except KeyError:
            pass
    return moves

assert move(automaton, [1], 'a') == set()
assert move(automaton, [2], 'a') == {4}
assert move(automaton, [1, 5], 'd') == {5}

Pasemos a computar $\epsilon$-closure`(T)`. El método `epsilon_closure(automaton, states)` computa el conjunto $\epsilon$-closure`(states)` con las transiciones registradas en `automaton`.

In [22]:
def epsilon_closure(automaton, states):
    pending = [ s for s in states ] # equivalente a list(states) pero me gusta así :p
    closure = { s for s in states } # equivalente a  set(states) pero me gusta así :p
    
    while pending:
        state = pending.pop()
        for dest in automaton.epsilon_transitions(state):
            if dest not in closure:
                closure.add(dest)
                pending.append(dest)
                
    return ContainerSet(*closure)

assert epsilon_closure(automaton, [0]) == {0,1,2,3}
assert epsilon_closure(automaton, [0, 4]) == {0,1,2,3,4,5}
assert epsilon_closure(automaton, [1, 2, 4]) == {1,2,3,4,5}

Finalmente implementemos el algoritmo para convertir de autómata finito no determinista a determinista.

In [28]:
def nfa_to_dfa(automaton):
    transitions = {}
    
    start = epsilon_closure(automaton, [automaton.start])
    start.id = 0
    start.is_final = any(s in automaton.finals for s in start)
    states = [ start ]

    pending = [ start ]
    while pending:
        state = pending.pop()
        
        for symbol in automaton.vocabulary:
            destinations = move(automaton, state, symbol)
            destinations = epsilon_closure(automaton, destinations)
            if len(destinations) == 0:
                continue
            if destinations not in states:
                destinations.is_final = any(
                    s in automaton.finals for s in destinations)
                destinations.id = len(states)
                states.append(destinations)
                pending.append(destinations)
            else:
                destinations = states[states.index(destinations)]

            try:
                transitions[state.id, symbol]
                assert False, 'Invalid DFA!!!'
            except KeyError:
                transitions[state.id, symbol] = destinations.id
            except AssertionError:
                pass
    
    finals = [ state.id for state in states if state.is_final ]
    dfa = DFA(len(states), finals, transitions)
    return dfa

Comprobemos que funciona el algoritmo. El DFA resultante debería reconocer el mismo lenguaje que el NFA original: `c*|(a|b)d*`.

In [29]:
dfa = nfa_to_dfa(automaton)
display(dfa)

assert dfa.states == 4
assert len(dfa.finals) == 4

assert dfa.recognize('')
assert dfa.recognize('a')
assert dfa.recognize('b')
assert dfa.recognize('cccccc')
assert dfa.recognize('adddd')
assert dfa.recognize('bdddd')

assert not dfa.recognize('dddddd')
assert not dfa.recognize('cdddd')
assert not dfa.recognize('aa')
assert not dfa.recognize('ab')
assert not dfa.recognize('ddddc')

![](cp6-files/3.svg)

### Probando con otros autómatas

Qué lenguaje reconoce el siguiente autómata?

In [30]:
automaton = NFA(states=3, finals=[2], transitions={
    (0,'a'): [ 0 ],
    (0,'b'): [ 0, 1 ],
    (1,'a'): [ 2 ],
    (1,'b'): [ 2 ],
})
display(automaton)

print("No sé que lenguaje reconoce :'(")


assert move(automaton, [0, 1], 'a') == {0, 2}
assert move(automaton, [0, 1], 'b') == {0, 1, 2}

dfa = nfa_to_dfa(automaton)
display(dfa)

assert dfa.states == 4
assert len(dfa.finals) == 2

assert dfa.recognize('aba')
assert dfa.recognize('bb')
assert dfa.recognize('aaaaaaaaaaaba')

assert not dfa.recognize('aaa')
assert not dfa.recognize('ab')
assert not dfa.recognize('b')
assert not dfa.recognize('')

No sé que lenguaje reconoce :'(


![](cp6-files/4.svg)
![](cp6-files/5.svg)

Y este?

In [31]:
automaton = NFA(states=5, finals=[4], transitions={
    (0,'a'): [ 0, 1 ],
    (0,'b'): [ 0, 2 ],
    (0,'c'): [ 0, 3 ],
    (1,'a'): [ 1, 4 ],
    (1,'b'): [ 1 ],
    (1,'c'): [ 1 ],
    (2,'a'): [ 2 ],
    (2,'b'): [ 2, 4 ],
    (2,'c'): [ 2 ],
    (3,'a'): [ 3 ],
    (3,'b'): [ 3 ],
    (3,'c'): [ 3, 4 ],
})
display(automaton)

print("No sé que lenguaje reconoce :'(")

dfa = nfa_to_dfa(automaton)
display(dfa)

assert dfa.states == 15
assert len(dfa.finals) == 7

assert dfa.recognize('abccac')
assert dfa.recognize('bbbbbbbbaa')
assert dfa.recognize('cac')

assert not dfa.recognize('abbbbc')
assert not dfa.recognize('a')
assert not dfa.recognize('')
assert not dfa.recognize('acacacaccab')

No sé que lenguaje reconoce :'(


![](cp6-files/6.svg)
![](cp6-files/7.svg)

Se le ocurre un autómata determinista para este lenguaje sin pasar por la transformación? 

## Propuestas

- Implemente un algoritmo para reconocer si una cadena pertenece o no al lenguaje denotado por un autómata finito no determinista. Evite utilizar `backtrack` o transformar completamente el NFA a DFA. 

In [ ]:
def recognize(nfa, string):
    current = epsilon_closure(nfa, [nfa.start])
    for symbol in string:
        current = epsilon_closure(nfa, move(nfa, current, symbol))
    return any(s in nfa.finals for s in current)

In [ ]:
automaton = NFA(states=5, finals=[4], transitions={
    (0,'a'): [ 0, 1 ],
    (0,'b'): [ 0, 2 ],
    (0,'c'): [ 0, 3 ],
    (1,'a'): [ 1, 4 ],
    (1,'b'): [ 1 ],
    (1,'c'): [ 1 ],
    (2,'a'): [ 2 ],
    (2,'b'): [ 2, 4 ],
    (2,'c'): [ 2 ],
    (3,'a'): [ 3 ],
    (3,'b'): [ 3 ],
    (3,'c'): [ 3, 4 ],
})

assert recognize(automaton, 'abccac')
assert recognize(automaton, 'bbbbbbbbaa')
assert recognize(automaton, 'cac')

assert not recognize(automaton, 'abbbbc')
assert not recognize(automaton, 'a')
assert not recognize(automaton, '')
assert not recognize(automaton, 'acacacaccab')